<a href="https://colab.research.google.com/github/al34n1x/CSA_LATAM2018/blob/master/6.Gestion_de_datos/Agregaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agregación de datos y operaciones de grupo

La categorización de un conjunto de datos y la aplicación de una función a cada grupo, ya sea una agregación o transformación, es un componente crítico del trabajo de análisis de datos. Después de cargar, fusionar y preparar un conjunto de datos, es posible que debas calcular estadísticas de grupo o posiblemente tablas dinámicas para fines de informes o visualización. Pandas proporciona una interfaz de grupo flexible, que te permite cortar, cortar en dados y resumir conjuntos de datos de forma natural.

Como verás, con la expresividad de Python y pandas, podemos realizar operaciones grupales bastante complejas utilizando cualquier función que acepte un objeto pandas o una matriz NumPy. 

## Actividades que veremos en este apartado

* Dividir un objeto pandas en pedazos usando una o más claves (en forma de funciones, matrices o nombres de columna de DataFrame)

* Calcular estadísticas de resumen de grupo, como conteo, media o desviación estándar, o una función definida por el usuario

* Aplique transformaciones dentro del grupo u otras manipulaciones, como normalización, regresión lineal, clasificación o selección de subconjuntos

* Calcular tablas dinámicas y tabulaciones cruzadas

* Realizar análisis cuantiles y otros análisis de grupos estadísticos.

## Mecánica del GroupBy

Existe un término conocido entre los analistas que describe operaciones de grupo,*split-apply-combine* 
En la primera parte de este proceso dividimos (split) dataframes o series en grupos basados en una o más keys. Una vez realizado la división, realizamos la función *apply* a cada grupo, produciendo un nuevo valor. 
Finalmente, tomamos el resultado de esas operaciones y las combinamos en un objeto.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*source: Python for Data Analysis, 2nd Edition*



In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.036122,0.028197
1,a,two,-0.182504,0.580248
2,b,one,-0.136171,-1.355133
3,b,two,-0.977995,0.209023
4,a,one,-1.357029,-1.010761


Supongamos que deseas calcular la media *(mean())* de la columna data1 usando las etiquetas de key1

In [0]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [0]:
grouped.mean()

Aquí agrupamos los datos usando dos claves, y la Serie resultante ahora tiene un índice jerárquico.

In [0]:
media = df['data1'].groupby([df['key1'], df['key2']]).mean()
media

key1  key2
a     one    -0.319943
      two     0.546181
b     one    -2.620541
      two     0.987506
Name: data1, dtype: float64

In [0]:
media.unstack()

En el siguiente ejemplo todo el grupo de keys son series, de esta manera podría haber cualquier arreglo de la longitud correcta 

In [0]:
prov = np.array(['Buenos Aires', 'Buenos Aires', 'Córdoba', 'Córdoba', 'Tucumán'])
anios = np.array([2005, 2005, 2006, 2005, 2006])

In [0]:
df['data1'].groupby([prov, anios]).mean()

## Seleccionando una columna o subset de columnas

La indexación de un objeto **GroupBy** creado a partir de un DataFrame con un nombre de columna o matriz de nombres de columna, genera un subconjunto de columnas para la agregación.

In [0]:
df.groupby('key1')['data1'] # Equivalente a df['data1'].groupby(df['key1'])
df.groupby('key1')[['data2']] # Equivalente a df[['data2']].groupby(df['key1'])

Especialmente para grandes conjuntos de datos, puede ser conveniente agregar solo unas pocas columnas. Por ejemplo, en el conjunto de datos anterior, para calcular medios solo para la columna data2 y obtener el resultado como un DataFrame, podríamos escribir:

In [0]:
df.groupby(['key1', 'key2'])[['data2']].mean()

El objeto devuelto por esta operación de indexación es un DataFrame agrupado si se pasa una lista o matriz o una Serie agrupada si solo se pasa un solo nombre de columna como escalar

In [0]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one     0.625985
      two     1.098123
b     one     1.264394
      two    -0.223157
Name: data2, dtype: float64

## Agrupando con dicts y series

Puede que necesites agrupar información existente en algo diferente a un arreglo. Consideremos el siguiente Dataframe:


In [0]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [5]:
people.iloc[2:3, [1, 2]] = np.nan # Agrega un par de NaN
people

,a,b,c,d,e
Joe,-0.209775,0.113849,-0.746741,1.744495,0.582583
Steve,0.218053,-1.634779,0.537866,1.551900,-1.397482
Wes,-0.017142,NaN,NaN,0.594722,2.115985
Jim,0.153630,1.121073,0.913158,-1.988261,0.487276
Travis,0.417668,1.383748,-0.325849,0.871233,0.540269


Supongamos que tenemos una lista de columnas que corresponden a ese dataframe y queremos realizar una operacion **sum** entre las columnas por grupo

In [0]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

Ahora podemos construir un arreglo a partir del dict y se lo pasamos a la operación **groupby**, pero en cambio le pasamos directamente el dict como key.


In [0]:
by_column=people.groupby(mapping, axis=1)

In [0]:
by_column.sum()

## Agrupación con funciones
El uso de las funciones de Python es una forma más genérica de definir un mapeo de grupo en comparación con un dict o Series. Cualquier función que se pase como clave de grupo se llamará una vez por valor de índice, y los valores de retorno se utilizarán como nombres de grupo. Más concretamente, consideremos el DataFrame de ejemplo de la sección anterior, que tiene los nombres de las personas como valores de índice. Supongamos que deseas agrupar por la longitud de los nombres; Si bien podrías calcular una matriz de longitudes de cadena, es más simple simplemente pasar la función len:

In [9]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.073288,1.234922,0.166416,0.350955,3.185844
5,0.218053,-1.634779,0.537866,1.551900,-1.397482
6,0.417668,1.383748,-0.325849,0.871233,0.540269
